file_paths = [
    'verified-smart-contracts/test/part.0.parquet',
    'verified-smart-contracts/validation/part.0.parquet',
    'verified-smart-contracts/train/part.0.parquet',
    'verified-smart-contracts/train/part.1.parquet',
    'verified-smart-contracts/train/part.2.parquet',
    'verified-smart-contracts/train/part.3.parquet',
    'verified-smart-contracts/train/part.4.parquet',
]

df = pd.concat([pd.read_parquet(file) for file in file_paths], ignore_index=True)
results = df[df['source_code'].str.contains("block.number|block.timestamp")]
results = results.drop_duplicates(subset='contract_address', keep='first')
results.reset_index(drop=True, inplace=True)
results['contract_address'].to_csv('potential_tl_contract_addresses.csv', index_label='index')

In [2]:
import pandas as pd

In [3]:
# Connect to db
from sqlalchemy import create_engine, text
from utils.db import get_inspect_database_uri

conn = create_engine(get_inspect_database_uri()).connect()

In [6]:
query = f"SELECT * FROM contracts_info WHERE eth_balance > 0"
info_df = pd.DataFrame(conn.execute(text(query)))

In [7]:
info_df

,contract_address,eth_balance,largest_tx_hash,largest_tx_block_number,largest_tx_value
0,0xe1a66a7883578bfaca0bd312bf38f8d16c320775,9.000000e-02,0x688b3f2348da522ff92b1480a8fa92b2576bdbba545e...,16703726,0.000000e+00
1,0xf4f140a406d5749f40dab9634384b6941f50f8eb,2.138195e-09,0xea3f0b52d6a1da7f6d7247e9cbc900c015e98c59bcac...,16293133,0.000000e+00
2,0xa9356faaa6c2339c6b9feb93127ede736053a1b8,1.000000e-18,0x643351e59198922a30a32bddb690244fde734524f78b...,16133046,0.000000e+00
3,0xf18276bd2e5ec7279980c520160328da2e4a5042,1.000000e-18,0xdbabe32f7ea1fb49bfb42db0af2ed18a5aac062cdbd4...,16309979,0.000000e+00
4,0x37c99907d203e68b5c4932a0a94a041d7b736806,4.033495e-03,0x5c7b7d202b3faf4421163ca88a5ed132d7d3ce9ddfc1...,17695903,1.553129e+16
...,...,...,...,...,...
724,0xe4aeee528083a9146a8bac32aa59a681c420982a,1.501397e-02,0x181317d7e282e0d165760b2cf7449f1e497f432ecf54...,16561411,0.000000e+00
725,0xcda15caa30e3d9757530dc325f4e102b853af30c,1.000000e-18,0xca34b08ecc1109e59a4b7bf5bcfe1bad5723fd569edf...,16900973,0.000000e+00
726,0xbdba884842f9a56d3ffe31c7a59f03dbb9153883,3.900000e-04,0x105dfdec432e744e63a813fc2789879ff57a016f0cb7...,17553669,0.000000e+00
727,0x1ec1b3fffd5072d97b27110a667c35025c96d5c5,2.695338e-03,0x9348370a43647a4f3cbeaa6790072be66081774d69e4...,18349728,0.000000e+00


In [24]:
# get the 10 highest balance contracts
info_df.sort_values(by='eth_balance', ascending=False).head(50)

BASE_GAS_FEE = 11e9 # 11 Gwei
BLOCK_GAS_LIMIT = 30e6 # 30 million units
ETH_TO_WEI = 1e18 # 1 ETH = 1e18 WEI    
# get the contracts whose eth balance is higher than block gas limit * base gas price
info_df[info_df['eth_balance'] > ((BLOCK_GAS_LIMIT * BASE_GAS_FEE) / ETH_TO_WEI)]

,contract_address,eth_balance,largest_tx_hash,largest_tx_block_number,largest_tx_value
11,0x59d71082d8a5b18ebc6b653ae422ac4383cd2597,3.111485,0x3a39d1e9768ce3b54177a55a3dd1808cd2f41e6a82f5...,18011475,0.000000e+00
13,0x0b5da756938e334c97ce20715e32a4a8fea12ba9,1.520784,0xbb1bcda59d9e7555db25965e86d83bad14242e068fe7...,18354970,1.000000e+13
19,0x3f7aff0ef20aa2e646290dfa4e67611b2220c597,37.514262,0xec948a58790102d74d0c2cc8e1239b56ac2d989169b8...,18387365,0.000000e+00
30,0xfe5f69dfa2d4501e78078266f6d430c079098f90,8.715299,0xd28137f099a9798b4d51f22ac28712c739397c73baba...,18111034,0.000000e+00
33,0x3a275655586a049fe860be867d10cdae2ffc0f33,17.876120,0x2ee415e9937ec5eaf2a2cd5c753b5c7bc177edd84c30...,16621526,0.000000e+00
...,...,...,...,...,...
701,0x3428a5e2c789f637554d1612a8ef381efa266aca,0.351908,0xff858616f478bed6d581701ddd329a8e40c5f332c3ad...,16736168,0.000000e+00
702,0x3333333acdedbbc9ad7bda0876e60714195681c5,1.000000,0x7b200f7707a44910800281f4bba37ae1f16fe2c47797...,18364773,0.000000e+00
704,0xc4638af1e01720c4b5df3bc8d833db6be85d2211,1.904000,0x6ee8733d1aad89fcade8c15d01dcf4fb96e3155ea7fe...,18383765,5.000000e+16
714,0x53828420451bd4298c8b1b8ce5c8f3474ea85e3d,1.820000,0xe4c4d925fbabb2851d8803db236674a956f5b29a536d...,17965925,0.000000e+00


In [25]:
BASE_GAS_FEE * BLOCK_GAS_LIMIT / ETH_TO_WEI

0.33

In [10]:
info_df.describe()

,eth_balance,largest_tx_block_number,largest_tx_value
count,7.290000e+02,7.290000e+02,7.290000e+02
mean,1.121242e+02,1.731477e+07,6.131261e+15
std,2.334077e+03,8.712884e+05,6.524462e+16
min,1.000000e-18,1.565749e+07,0.000000e+00
25%,1.000000e-18,1.648061e+07,0.000000e+00
50%,2.246191e-02,1.745672e+07,0.000000e+00
75%,3.100000e-01,1.823383e+07,0.000000e+00
max,6.239257e+04,1.838854e+07,1.400000e+18


In [11]:
import requests

OLDEST_BLOCK = 15649595  # first block on October 2022
NEWEST_BLOCK = 18375639  # last block on October 2023
API_KEY = "T48DJNQHTTC6TGKR4ZW19ZSBR9XH17R5S3"
address = info_df.sort_values(by='eth_balance', ascending=False).iloc[0]['contract_address']

In [19]:
# send request to fetch transactions of the contract with the highest balance
url =  f"https://api.etherscan.io/api?module=account&action=txlist&address={address}&startblock={OLDEST_BLOCK}&endblock={NEWEST_BLOCK}&page=1&offset=10000&sort=desc&apikey={API_KEY}"

response = requests.get(url)

In [22]:
response.json()['result']

[{'blockNumber': '18375060',
  'timeStamp': '1697605727',
  'hash': '0xd70015353cb55fd426202af676d10ad2c5948e93e5e5c6ed716e34d091e148c4',
  'nonce': '382',
  'blockHash': '0x9f10a7b80a76e8d6372b704f2c3cac90d75b2e7603569842fc962307989538b9',
  'transactionIndex': '25',
  'from': '0xa3ef8ebe3e28f3e12141b6a72ae9bd85cdb29af0',
  'to': '0xdc24316b9ae028f1497c275eb9192a3ea0f67022',
  'value': '0',
  'gas': '200493',
  'gasPrice': '7098347212',
  'isError': '0',
  'txreceipt_status': '1',
  'input': '0x1a4d01d20000000000000000000000000000000000000000000000018634c4fb95f32fbb0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001a5ecf599d5569565',
  'contractAddress': '',
  'cumulativeGasUsed': '4411546',
  'gasUsed': '127331',
  'confirmations': '15064',
  'methodId': '0x1a4d01d2',
  'functionName': 'remove_liquidity_one_coin(uint256 _token_amount, int128 i, uint256 _min_amount)'},
 {'blockNumber': '18374460',
  'timeStamp': '1697598491'

In [26]:
# get the tx with the highest value
tx = max(response.json()['result'], key=lambda x: int(x['value']))
tx

{'blockNumber': '16653088',
 'timeStamp': '1676693723',
 'hash': '0x22404d240227bc9152b569dc89957b769a74407b12fb2c1b7122914415f59460',
 'nonce': '618',
 'blockHash': '0xee624d477e5e2dabeaeaa19b540a997b90379926e43150597c4cc300b3997ace',
 'transactionIndex': '13',
 'from': '0x9026a229b535ecf0162dfe48fdeb3c75f7b2a7ae',
 'to': '0xdc24316b9ae028f1497c275eb9192a3ea0f67022',
 'value': '46441988320787554000000',
 'gas': '706044',
 'gasPrice': '25000000000',
 'isError': '0',
 'txreceipt_status': '1',
 'input': '0x0b4c7e4d0000000000000000000000000000000000000000000009d5a00f2d8e764e5c80000000000000000000000000000000000000000000000a059d799f1f832ae7800000000000000000000000000000000000000000000012a1f92462a8dc6d6d10',
 'contractAddress': '',
 'cumulativeGasUsed': '985354',
 'gasUsed': '176511',
 'confirmations': '1737036',
 'methodId': '0x0b4c7e4d',
 'functionName': 'add_liquidity(uint256[2] amounts, uint256 min_mint_amount)'}